# LDA Hyperparameter Optimization

Этот ноутбук демонстрирует оптимизацию гиперпараметров LDA с использованием генетического алгоритма (GA) и эволюционной стратегии (ES).

## Методология

**Правильный подход к подбору гиперпараметров:**
1. Обучаем LDA на **validation** выборке
2. Оптимизируем для минимизации средней **perplexity** на той же validation выборке
3. Ищем оптимальное T (число топиков), alpha и eta устанавливаются как 1/T

In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

from lda_optimizer import (
    load_bow_data,
    GAOptimizer,
    ESOptimizer,
    ParallelRunner,
    make_objective,
    make_eval_func,
    run_single_optimization
)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json

sns.set_style('whitegrid')
print("Импорты успешно выполнены!")

## 1. Пример: Одиночная оптимизация на одном датасете

Начнем с простого примера - запустим GA на датасете 20news.

In [ ]:
# Загрузка данных
val_path = "data/X_20news_val_bow.npz"
Xval = load_bow_data(val_path)

print(f"Загружено: {Xval.shape[0]} документов, словарь из {Xval.shape[1]} слов")

In [ ]:
# Запуск GA оптимизации (с небольшим числом поколений для демонстрации)
summary = run_single_optimization(
    algorithm_name="GA",
    optimizer_class=GAOptimizer,
    Xval=Xval,
    outdir="logs/20news/ga_demo",
    dataset_name="20news",
    gens_or_steps=10,  # Небольшое число для быстрой демонстрации
    pop_size=5,
    seed=42,
    max_iter=30,  # Меньше итераций LDA для ускорения
    batch_size=2048,
    learning_method="online",
    elite=2,
    early_stop_eps_pct=0.01,
    max_no_improvement=3
)

print("\nЛучшие найденные параметры:")
print(json.dumps(summary['best'], indent=2))

## 2. Сравнение GA и ES на одном датасете

Запустим оба алгоритма последовательно и сравним результаты.

In [ ]:
# Параметры
dataset_name = "agnews"
val_path = f"data/X_{dataset_name}_val_bow.npz"
Xval = load_bow_data(val_path)

gens = 15
pop_size = 8
seed = 42
max_iter = 40

print(f"Датасет: {dataset_name}")
print(f"Размер: {Xval.shape[0]} документов, {Xval.shape[1]} слов")

In [ ]:
# Запуск GA
print("\n=" * 80)
print("ЗАПУСК GA")
print("=" * 80)

summary_ga = run_single_optimization(
    algorithm_name="GA",
    optimizer_class=GAOptimizer,
    Xval=Xval,
    outdir=f"logs/{dataset_name}/ga_compare",
    dataset_name=dataset_name,
    gens_or_steps=gens,
    pop_size=pop_size,
    seed=seed,
    max_iter=max_iter,
    elite=3,
    early_stop_eps_pct=0.01,
    max_no_improvement=3
)

In [ ]:
# Запуск ES
print("\n=" * 80)
print("ЗАПУСК ES")
print("=" * 80)

summary_es = run_single_optimization(
    algorithm_name="ES",
    optimizer_class=ESOptimizer,
    Xval=Xval,
    outdir=f"logs/{dataset_name}/es_compare",
    dataset_name=dataset_name,
    gens_or_steps=gens,
    pop_size=pop_size,
    seed=seed,
    max_iter=max_iter,
    mu=4,
    lmbda=8,
    early_stop_eps_pct=0.01,
    max_no_improvement=3
)

In [ ]:
# Сравнение результатов
comparison = pd.DataFrame([
    {
        'Algorithm': 'GA',
        'Best T': summary_ga['best']['T'],
        'Best Perplexity': summary_ga['best']['perplexity'],
        'Total Time (s)': summary_ga['total_time'],
        'Iterations': summary_ga['num_iterations']
    },
    {
        'Algorithm': 'ES',
        'Best T': summary_es['best']['T'],
        'Best Perplexity': summary_es['best']['perplexity'],
        'Total Time (s)': summary_es['total_time'],
        'Iterations': summary_es['num_iterations']
    }
])

print("\nСравнение алгоритмов:")
print(comparison.to_string(index=False))

## 3. Параллельный запуск на нескольких датасетах

Используем `ParallelRunner` для одновременного запуска оптимизации на нескольких датасетах.

In [ ]:
# Конфигурация для параллельного запуска
datasets_subset = {
    "20news": "data/X_20news_val_bow.npz",
    "agnews": "data/X_agnews_val_bow.npz"
}

# Параметры
gens = 10
pop_size = 6
seed = 42
max_iter = 30

ga_params = {
    "early_stop_eps_pct": 0.01,
    "max_no_improvement": 3,
    "elite": 2
}

es_params = {
    "early_stop_eps_pct": 0.01,
    "max_no_improvement": 3,
    "mu": 3,
    "lmbda": 6
}

# Создание задач
tasks = []
for name, val_path in datasets_subset.items():
    # GA task
    ga_task_params = {
        "outdir": f"logs/{name}/ga_parallel",
        "dataset_name": name,
        "gens_or_steps": gens,
        "pop_size": pop_size,
        "seed": seed,
        "max_iter": max_iter,
        "batch_size": 2048,
        "learning_method": "online",
        **ga_params
    }
    tasks.append((name, val_path, "GA", ga_task_params))
    
    # ES task
    es_task_params = {
        "outdir": f"logs/{name}/es_parallel",
        "dataset_name": name,
        "gens_or_steps": gens,
        "pop_size": pop_size,
        "seed": seed,
        "max_iter": max_iter,
        "batch_size": 2048,
        "learning_method": "online",
        **es_params
    }
    tasks.append((name, val_path, "ES", es_task_params))

print(f"Подготовлено {len(tasks)} задач для параллельного выполнения")
print("Задачи:")
for name, _, alg, _ in tasks:
    print(f"  - {alg} на {name}")

In [ ]:
# Запуск параллельных задач
runner = ParallelRunner(max_workers=4)
results = runner.run_all(tasks)

print("\n" + "="*80)
print("Все задачи завершены!")
print("="*80)
print(json.dumps(results, indent=2))

## 4. Визуализация результатов

Построим графики для сравнения производительности алгоритмов.

In [ ]:
# Загрузка результатов истории для визуализации
import os

def load_history(dataset, algorithm):
    """Загрузить историю оптимизации"""
    if algorithm.lower() == 'ga':
        path = f"logs/{dataset}/ga_compare/history.csv"
    else:
        path = f"logs/{dataset}/es_compare/history.csv"
    
    if os.path.exists(path):
        return pd.read_csv(path)
    return None

# Загрузка данных
hist_ga = load_history(dataset_name, 'ga')
hist_es = load_history(dataset_name, 'es')

if hist_ga is not None and hist_es is not None:
    print(f"История загружена для {dataset_name}")
    print(f"GA: {len(hist_ga)} итераций")
    print(f"ES: {len(hist_es)} итераций")
else:
    print("История не найдена. Запустите предыдущие ячейки сначала.")

In [ ]:
# График сравнения перплексии
if hist_ga is not None and hist_es is not None:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Perplexity
    ax = axes[0, 0]
    ax.plot(hist_ga['iter'], hist_ga['best_perplexity'], 'o-', label='GA', linewidth=2)
    ax.plot(hist_es['iter'], hist_es['best_perplexity'], 's-', label='ES', linewidth=2)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Best Perplexity')
    ax.set_title(f'{dataset_name}: Best Perplexity vs Iteration')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # T (Topics)
    ax = axes[0, 1]
    ax.plot(hist_ga['iter'], hist_ga['T_best'], 'o-', label='GA', linewidth=2)
    ax.plot(hist_es['iter'], hist_es['T_best'], 's-', label='ES', linewidth=2)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Best T')
    ax.set_title(f'{dataset_name}: Best T vs Iteration')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Population mean
    ax = axes[1, 0]
    ax.plot(hist_ga['iter'], hist_ga['pop_mean'], 'o-', label='GA', linewidth=2)
    ax.plot(hist_es['iter'], hist_es['pop_mean'], 's-', label='ES', linewidth=2)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Population Mean Fitness')
    ax.set_title(f'{dataset_name}: Population Mean vs Iteration')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # Cumulative time
    ax = axes[1, 1]
    ax.plot(hist_ga['iter'], hist_ga['cum_time'], 'o-', label='GA', linewidth=2)
    ax.plot(hist_es['iter'], hist_es['cum_time'], 's-', label='ES', linewidth=2)
    ax.set_xlabel('Iteration')
    ax.set_ylabel('Cumulative Time (s)')
    ax.set_title(f'{dataset_name}: Cumulative Time vs Iteration')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(f'logs/{dataset_name}/comparison_plot.png', dpi=150)
    plt.show()
    
    print(f"График сохранен в logs/{dataset_name}/comparison_plot.png")

## 5. Итоговая таблица результатов

Соберем все результаты в одну таблицу для удобства.

In [ ]:
# Создание итоговой таблицы
summary_data = []

if hist_ga is not None:
    summary_data.append({
        'Dataset': dataset_name,
        'Algorithm': 'GA',
        'Best T': summary_ga['best']['T'],
        'Best Perplexity': f"{summary_ga['best']['perplexity']:.4f}",
        'Iterations': summary_ga['num_iterations'],
        'Total Time (s)': f"{summary_ga['total_time']:.2f}",
        'Avg Step Time (s)': f"{summary_ga['avg_step_time']:.2f}",
        'Early Stopped': summary_ga['stopped_early']
    })

if hist_es is not None:
    summary_data.append({
        'Dataset': dataset_name,
        'Algorithm': 'ES',
        'Best T': summary_es['best']['T'],
        'Best Perplexity': f"{summary_es['best']['perplexity']:.4f}",
        'Iterations': summary_es['num_iterations'],
        'Total Time (s)': f"{summary_es['total_time']:.2f}",
        'Avg Step Time (s)': f"{summary_es['avg_step_time']:.2f}",
        'Early Stopped': summary_es['stopped_early']
    })

summary_df = pd.DataFrame(summary_data)
print("\nИтоговые результаты:")
print(summary_df.to_string(index=False))

# Сохранение в CSV
summary_df.to_csv(f'logs/{dataset_name}/summary_table.csv', index=False)
print(f"\nТаблица сохранена в logs/{dataset_name}/summary_table.csv")

## Заключение

В этом ноутбуке мы продемонстрировали:
1. **Одиночную оптимизацию** с использованием GA
2. **Сравнение GA и ES** на одном датасете
3. **Параллельный запуск** с использованием `ParallelRunner`
4. **Визуализацию результатов** для анализа производительности

### Основные выводы:
- Оба алгоритма (GA и ES) эффективно находят хорошие гиперпараметры
- Параллельный запуск значительно ускоряет процесс оптимизации
- Обучение на validation set с оценкой perplexity - правильный подход

### Для полного запуска на всех датасетах:
```bash
# Последовательно
python lda_optimizer.py

# Параллельно
python lda_optimizer.py --parallel

# Только GA
python lda_optimizer.py --algorithm ga

# GA параллельно
python lda_optimizer.py --parallel --algorithm ga
```